# Neural Networks




## Neural Networks

&ensp; &ensp; &ensp; &ensp; **Neural networks (NNs)** are computational models inspired by the structure of the human brain, designed to recognize patterns and make predictions. They consist of layers of interconnected nodes (often called neurons) that process information through mathematical operations.

A basic neural network has the following structure:

1. **Input Layer**: It receives raw data, like pixels of an image, or tokens of a text. Each node in this layer represents an input dimension.
2. **Hidden Layers**: They are between the input layer and output layer, and process the information. Each hidden layer transforms data from the previous layer, allowing the network to progressively learn and recognize patterns.
3. **Output Layer**: It provides the network’s output.

```{figure} ../images/neural-network.png
---
width: 300px
name: neural-network
---
Basic Structure of a Neural Network
```



(2.2)=
## Neurons

&ensp; &ensp; &ensp; &ensp; A **neuron** is a unit that takes in multiple inputs and processes them to produce a single output. As shown above in [*Fig. 1.2*](neural-network), each neuron in the hidden and output layers connects to all the neurons in the previous layer. These connections have associated values, known as **weights**, which are adjusted by the model. The weights represent the strength of connection between the neurons. A greater weight indicates a stronger connection.


```{figure} ../images/neuron.png
---
width: 220px
name: neuron
---
Structure of a Neuron
```


```{admonition} Neuron's output

To calculate the output of a neuron, all the inputs to the neuron ($x_{1}, x_{2}, \dots, x_{d}$) are multiplied by their corresponding connection weights ($w_{1}, w_{2}, \dots, w_{d}$), and the products are summed. 

A numerical value, called the bias ($b$), is then added to the weighted sum. Finally, the result is passed through an **activation function** ($\sigma$) that returns the output of the neuron, the neuron's **activation value** ($h$).

<br>

$$
\small
h = \sigma\left(x_1w_1 + x_2w_2 + \dots + x_dw_d + b\right)
$$

where:

- $d$ is the dimensionality of the input vector.
```


```{important}
Please note that the ouput of a neuron is one of the inputs to all the neurons in the next layer.
```




## How NNs Learn

Neural networks use supervised learning to adjust their parameters (weights and biases) to minimize the loss function. This process involves repeating these four steps during training:

1. **Forward Pass:** Process input data though network and generate predictions.
2. **Calculate Loss:** Calculate the error of the predictions.
3. **Backward Pass:** Compute gradient of the loss function with respect to the network's parameters.
4. **Update Parameters:** Adjust network's parameters to minimize the loss.




## Digits Recognizer

&ensp; &ensp; &ensp; &ensp; In this and the following chapters, we will build a discriminative model to recognize handwritten digits. To train our neural network, we will use the [MNIST dataset](https://huggingface.co/datasets/ylecun/mnist), which contains 60,000 training images and 10,000 test images. All the images are labeled, grayscale, and 28 x 28 pixels.


```{figure} ../images/digit-recognizer.png
---
width: 400px
name: digit-recognizer
---
Digit Recognizer
```

In [1]:
import numpy as np
import torch

# convert PIL image to normalized PyToch tensor
def image_to_tensor(image):
    return torch.tensor(np.array(image)) / 255.0


def preprocess_data(split):
    
    x = []  # list to store image tensors
    y = []  # list to store labels

    for example in split:
        x.append(image_to_tensor(example['image']))
        y.append(example['label'])
    
    return torch.stack(x), torch.tensor(y)

In [2]:
# import libraries
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
import seaborn as sns
from datasets import load_dataset

# set print options for PyTorch tensors
torch.set_printoptions(linewidth=140, sci_mode=False, precision=4)

# load the MNIST dataset
ds = load_dataset("ylecun/mnist")

# preprocess the data (we will see this function in Chapter 4)
train_x, train_y = preprocess_data(ds['train'])

print(f"Images: {train_x.shape}")
print(f"Labels: {train_y.shape}")

Images: torch.Size([60000, 28, 28])
Labels: torch.Size([60000])


Each image is currently stored as a matrix of shape (28, 28). To be able to process these images, we need to flatten them into vectors of shape (784).

```{admonition} Help
:class: dropdown
`tensor.view()` efficiently reshapes the shape of a tensor.
```

In [1]:
# flatten images
X = train_x.view(-1, 784)

print(f"Images: {X.shape}")

NameError: name 'train_x' is not defined

## MLP

&ensp; &ensp; &ensp; &ensp; A **multilayer perceptron (MLP)** is a type of neural network composed of multiple layers of fully connected neurons with nonlinear activation functions. Our MLP will have 784 input neurons (one for each pixel in the image) and 10 output neurons (one for each possible class: 0 to 9).


```{figure} ../images/MLP.png
---
width: 400px
name: MLP
---
MLP
```


```{admonition} Help
:class: dropdown
`torch.randn()` generates a tensor filled with random numbers drawn from a normal distribution (mean = 0, standard deviation = 1).
```

In [7]:
def initialize_nn(n_hidden = 100):

    # seed for reproducibility
    g = torch.Generator().manual_seed(1)
    
    W1 = torch.randn((784, n_hidden),      generator=g)
    b1 = torch.zeros(n_hidden)
    W2 = torch.randn((n_hidden, n_hidden), generator=g)
    b2 = torch.zeros(n_hidden)
    W3 = torch.randn((n_hidden, 10),       generator=g)
    b3 = torch.zeros(10)

    parameters = [W1, b1, W2, b2, W3, b3]
    
    return parameters

parameters = initialize_nn()
W1, b1, W2, b2, W3, b3 = parameters

## Forward Pass

In the **forward pass**, the input data flows through the neural network, layer by layer, to produce the network's output.


```{admonition} Neuron's output

In section [2.2](2.2), we saw that the output of a neuron is given by the formula:

<br>

$$
\small
h = \sigma\left(x_1w_1 + x_2w_2 + \dots + x_dw_d + b\right)
$$

where:

- $d$ is the dimensionality of the input vector.
```


```{admonition} Neuron's output (dot product)

Using a dot product we can express the weighted sum as:

<br>

$$
\small
h = \sigma\left(
\begin{bmatrix}
x_1 & x_2 & \dots & x_d
\end{bmatrix}
\cdot
\begin{bmatrix}
w_1 \\
w_2 \\
\vdots \\
w_d
\end{bmatrix}
+ b\right)
$$
```


````{admonition} Layer's output (single examples)

Using a weight matrix we can calculate the activation values of all the neurons in the layer:

<br>

$$
\small
\begin{bmatrix}
h_1 & h_2 & \dots & h_m
\end{bmatrix}
=
\sigma\left(
\begin{bmatrix}
x_1 & x_2 & \dots & x_d
\end{bmatrix}
\cdot
\begin{bmatrix}
w_{11} & w_{12} & \dots & w_{1m} \\
w_{21} & w_{22} & \dots & w_{2m} \\
\vdots & \vdots & \ddots & \vdots \\
w_{d1} & w_{d2} & \dots & w_{dm} \\
\end{bmatrix}
+
\begin{bmatrix}
b_1 & b_2 & \dots & b_m
\end{bmatrix}
\right)
$$

where:

- $m$ is the number of neurons in the layer.


```{important}
Each column of the weight matrix contains the weights of the connections between a single neuron in the current layer and all the neurons in the previous layer.
```

````


````{admonition} Layer's output (multiple examples)

Uisng matrix multiplication we can efficiently calculate in parallel the output of a layer for several input examples:

<br>

$$
\small
\begin{bmatrix}
h_{11} & h_{12} & \dots & h_{1m} \\
h_{21} & h_{22} & \dots & h_{2m} \\
\vdots & \vdots & \ddots & \vdots \\
h_{N1} & h_{N2} & \dots & h_{Nm}
\end{bmatrix}
=
\sigma\left(
\begin{bmatrix}
x_{11} & h_{12} & \dots & h_{1d} \\
x_{21} & h_{22} & \dots & h_{2d} \\
\vdots & \vdots & \ddots & \vdots \\
x_{N1} & h_{N2} & \dots & h_{Nd}
\end{bmatrix}
\times
\begin{bmatrix}
w_{11} & w_{12} & \dots & w_{1m} \\
w_{21} & w_{22} & \dots & w_{2m} \\
\vdots & \vdots & \ddots & \vdots \\
w_{d1} & w_{d2} & \dots & w_{dm} \\
\end{bmatrix}
+
\begin{bmatrix}
b_{1} & b_{2} & \dots & b_{m}
\end{bmatrix}
\right)
$$

where:

- $N$ is the number of input examples.


```{important}
Each row of the output matrix contains the activation values of all the neurons in the layer for a single input example.
```

````


## Tanh


## Softmax

&ensp; &ensp; &ensp; &ensp; **Softmax** is an activation function often used in the output layer of neural networks. It transforms raw neural network outputs, known as logits, into probability distributions where each probability represents the model's confidence that a given example belongs to a specific class.

```{figure} ../images/softmax.png
---
width: 500px
name: softmax
---
Softmax
```

In [16]:
# forward pass
h1 = torch.tanh(X @ W1 + b1)   # (60000, 100) = (60000, 784) x (784, 100) + (100)
h2 = torch.tanh(h1 @ W2 + b2)  # (60000, 100) = (60000, 100) x (100, 100) + (100)
logits = h2 @ W3 + b3          # (60000,  10) = (60000, 100) x (100,  10) + (10)

# softmax
probs = logits.exp() / logits.exp().sum(1, keepdims=True)  # (60000,  10)

# print  first 5 probabilities
print(probs[:5])

tensor([[    0.6741,     0.3209,     0.0000,     0.0000,     0.0000,     0.0000,     0.0000,     0.0050,     0.0000,     0.0000],
        [    0.0000,     0.0000,     0.0000,     0.0000,     0.0000,     0.0000,     0.0000,     0.8520,     0.0000,     0.1480],
        [    0.0000,     1.0000,     0.0000,     0.0000,     0.0000,     0.0000,     0.0000,     0.0000,     0.0000,     0.0000],
        [    0.0000,     0.0000,     0.0005,     0.0000,     0.9965,     0.0000,     0.0000,     0.0001,     0.0028,     0.0000],
        [    0.9911,     0.0000,     0.0033,     0.0000,     0.0000,     0.0000,     0.0056,     0.0000,     0.0000,     0.0000]],
       grad_fn=<SliceBackward0>)


## Calculate Loss

&ensp; &ensp; &ensp; &ensp; The forward pass can also be seen as the step where the model, using its current parameters, generates predictions. The **Cross-Entropy Loss** is a widely used loss function for classification tasks that evaluates how well these predictions align with the labels. It combines Softmax with the average negative log-likelihood.

```{admonition} Likelihood

The **likelihood** represents the joint probability of the model assigning correct labels to all examples:

<br>

$$
\text{likelihood} = \prod_{i=1}^{N} p_i
$$

where:  
- $p_i$ is the probability assigned by the model to the correct class for the $i$-th example.  
- $N$ is the total number of examples in the dataset.
```

```{admonition} Log Likelihood

Since each $p_i$ is a value between 0 and 1, their product can become very small. To avoid numerical instability, we take the logarithm of the likelihood:

<br>

$$
\text{log likelihood} = \log \left(\prod_{i=1}^{N} p_i \right) = \sum_{i=1}^{N} \log(p_i)
$$
```


````{admonition} Negative Log Likelihood

Looking at the graph of the logarithmic function, please note that:

- If we pass in a probability of $1$, the log probability is $0$.
- If we pass in a lower probability $\left(0 < p < 1 \right)$, the log probability becomes more negative.  
- If we pass in a probability of 0, the log probability is $-\infty$.

```{figure} ../images/log-function.png
---
width: 200px
name: log-function
---
Logarithmic Function
```

<br>

Thus, when all the individual probabilities are 1 (the best-case scenario), the log likelihood is 0, and when the probabilities decrease, the log likelihood becomes more negative. To make it eassier to interpret, we use the **negative log likelihood (NLL)**, a positive metric where values closer to 0 indicate better predictions:

<br>

$$
\text{NLL} = - \sum_{i=1}^{N} \log(p_i)
$$

````

```{admonition} Average Negative Log Likelihood

To normalize the NLL, it is often divided by the total number of examples in the dataset:

<br>

$$
\text{Average NLL} = - \frac{1}{N} \sum_{i=1}^{N} \log(p_i)
$$

```

In [17]:
# number of train images 
N = 60000

# average NLL
loss = -probs[range(N), train_y].log().mean()

print(f"Loss: {loss:.4f}")

Loss: 15.9699


## Backward Pass

&ensp; &ensp; &ensp; &ensp; A lower loss indicates that the model is making more accurate predictions by assigning higher probabilities to the correct classes. Thus, to improve the model's performance, we aim to minimize the loss by adjusting its parameters. 

&ensp; &ensp; &ensp; &ensp; During the **backward pass**, or backpropagation, we calculate the gradient of the loss function with respect to these parameters. This gradient is determined by computing the derivative of the loss with respect to the model's parameters using the chain rule. To simplify this process, we will break the forward pass into smaller components, making it easier to differentiate.

In [ ]:
# hidden layer 1
h1_pre = X @ W1 + b1
h1 = torch.tanh(h1_pre)

# hidden layer 2
h2_pre = h1 @ W2 + b2
h2 = torch.tanh(h2_pre)

# output layer
logits = h2 @ W3 + b3

# softmax
counts = logits.exp()
counts_sum = counts.sum(1, keepdims=True)
counts_sum_inv = counts_sum**-1
probs = counts * counts_sum_inv

# average NLL
log_probs = probs.log()
loss = -log_probs[range(N), train_y].mean()

We first calculate the derivative of the loss with respect to the log probabilities.

<br>

$$
loss = - mean(log\_probs) \quad \Rightarrow \quad
\begin{matrix}
dlog\_probs = 0 \text{ if not p_i}\\
dlog\_probs = -\frac{1}{N} \text{if p_i}
\end{matrix}
$$

In [42]:
dlog_probs = torch.zeros_like(log_probs)
dlog_probs[range(N), train_y] = -1.0 / N

Next, we calculate the derivative of the loss with respect to the probabilities using the chain rule.

<br>

$$
log\_probs = log(probs) \quad \Rightarrow \quad dprobs = \frac{1}{probs} \cdot dlog\_probs
$$

In [43]:
dprobs = (1.0 / probs) * dlog_probs

We will continue backpropagating by calculating the derivative of the loss with respect to the intermediate values, and then, with respect to the model's parameters.

<br>

$$
probs = counts \cdot counts\_sum\_inv \quad \Rightarrow \quad
\begin{matrix}
dcounts = counts\_sum\_inv \cdot dprobs \\
dcounts\_sum\_inv = counts \cdot dprobs
\end{matrix}
$$

In [44]:
dcounts = counts_sum_inv * dprobs
dcounts_sum_inv = (counts * dprobs).sum(1, keepdim=True)

$$
counts\_sum\_inv = \frac{1}{counts\_sum} \quad \Rightarrow \quad dcounts\_sum = -\frac{1}{\sqrt{counts\_sum}} \cdot dcounts\_sum\_inv
$$

In [45]:
dcounts_sum = (-counts_sum**-2) * dcounts_sum_inv

$$
counts\_sum = \text{sum of rows of counts} \quad \Rightarrow \quad dcounts = dcounts\_sum
$$

In [46]:
dcounts += torch.ones_like(counts) * dcounts_sum

$$
counts = e^{logits} \quad \Rightarrow \quad dlogits = counts \cdot dcounts
$$

In [47]:
dlogits = counts * dcounts

$$
\text{logits} = h2 \times W3 + b3 \quad \Rightarrow \quad
\begin{matrix}
dh2 = dlogits \times (W3)^T \\
dW3 = (h2)^T \times dlogits \\
db3 = \text{sum of columns of dlogits}
\end{matrix}
$$

In [48]:
dh2 = dlogits @ W3.T
dW3 = h2.T @ dlogits
db3 = dlogits.sum(0)

$$
h2 = tanh(h2\_pre) \quad \Rightarrow \quad dh2\_pre = (1 - (h2)^2) \cdot dh2
$$

In [49]:
dh2_pre = (1.0 - h2**2) * dh2

$$
h2\_pre = h1 \times W2 + b2 \quad \Rightarrow \quad
\begin{matrix}
dh1 = dh2\_pre \times (W2)^T \\
dW2 = (h1)^T \times dh2\_pre \\
db2 = \text{sum of columns of dh2\_pre}
\end{matrix}
$$

In [50]:
dh1 = dh2_pre @ W2.T
dW2 = h1.T @ dh2_pre
db2 = dh2_pre.sum(0)

$$
h1 = tanh(h1\_pre) \quad \Rightarrow \quad dh1\_pre = (1 - (h1)^2) \cdot dh1
$$

In [51]:
dh1_pre = (1.0 - h1**2) * dh1

$$
h1\_pre = X \times W1 + b1 \quad \Rightarrow \quad
\begin{matrix}
dh1 = dh1\_pre \times (W1)^T \\
dW1 = (X)^T \times dh1\_pre \\
db1 = \text{sum of columns of dh1\_pre}
\end{matrix}
$$

In [52]:
dX = dh1_pre @ W1.T
dW1 = X.T @ dh1_pre
db1 = dh1_pre.sum(0)

We will create a list that contains the gradient of the loss function with respect to the parameters. 

In [ ]:
grads = [dW1, db1, dW2, db2, dW3, db3]

## Update Parameters

&ensp; &ensp; &ensp; &ensp; A gradient is a vector that indicates the direction and rate of the steepest increase in a function's value. To minimize the loss function, we use **gradient descent**, an optimization algorithm that updates the model's parameters by moving in the opposite direction of the gradient.

```{admonition} Gradient descent

Gradient descent updates each parameter by subtracting a fraction of the gradient from its current value, scaled by a factor known as the learning rate. 

<br>

$$
\theta = \theta - \eta \cdot \nabla L(\theta)
$$

where:
- $\theta$ represents the parameters of the neural network.
- $\eta$ is the learning rate.
- $\nabla L(\theta)$ is the gradient of the loss function with respect to the parameters.
```


```{important}
The learning rate determines the size of the steps we take towards the minimum. A smaller learning rate results in more precise but slower convergence, while a larger learning rate can speed up convergence but may risk overshooting the minimum.
```

In [ ]:
lr = 0.1 
for p, grad in zip(parameters, grads):
    p.data += -lr * grad

## Training

&ensp; &ensp; &ensp; &ensp; During **training**, a neural network iteratively performs a forward pass, calculates the loss, makes a backward pass, and updates its parameters. In this case, since the entire trainig dataset is processed in each training iteration, we can refer to each iteration as an epoch.

In [61]:
epochs = 100       # train iterations
lr = 0.1           # learning rate

# intialize neural network
parameters = initialize_nn()
W1, b1, W2, b2, W3, b3 = parameters

for epoch in range(epochs):

    # -------------------- forward pass --------------------

    # hidden layer 1
    h1_pre = X @ W1 + b1
    h1 = torch.tanh(h1_pre)

    # hidden layer 2
    h2_pre = h1 @ W2 + b2
    h2 = torch.tanh(h2_pre)

    # output layer
    logits = h2 @ W3 + b3

    # softmax
    counts = logits.exp()
    counts_sum = counts.sum(1, keepdims=True)
    counts_sum_inv = counts_sum**-1
    probs = counts * counts_sum_inv


    # -------------------- calculate loss --------------------

    # average negative log liklihood
    log_probs = probs.log()
    loss = -log_probs[range(N), train_y].mean()

    # print loss every 10 epochs
    if epoch % 10 == 0 or epoch == epochs - 1:
        print(f"Epoch: {epoch:2d}/{epochs}     Loss: {loss.item():.4f}")
    

    # -------------------- backward pass --------------------

    dlog_probs = torch.zeros_like(log_probs)
    dlog_probs[range(N), train_y] = -1.0 / N

    dprobs = (1.0 / probs) * dlog_probs
    dcounts = counts_sum_inv * dprobs
    dcounts_sum_inv = (counts * dprobs).sum(1, keepdim=True)
    dcounts_sum = (-counts_sum**-2) * dcounts_sum_inv
    dcounts += torch.ones_like(counts) * dcounts_sum

    dlogits = counts * dcounts
    dh2 = dlogits @ W3.T
    dW3 = h2.T @ dlogits
    db3 = dlogits.sum(0)
    dh2_pre = (1.0 - h2**2) * dh2
    dh1 = dh2_pre @ W2.T
    dW2 = h1.T @ dh2_pre
    db2 = dh2_pre.sum(0)
    dh1_pre = (1.0 - h1**2) * dh1
    dX = dh1_pre @ W1.T
    dW1 = X.T @ dh1_pre
    db1 = dh1_pre.sum(0)

    grads = [dW1, db1, dW2, db2, dW3, db3]


    # -------------------- update parameters --------------------

    for p, grad in zip(parameters, grads):
        p.data += -lr * grad

Epoch:  0/100     Loss: 15.9699
Epoch: 10/100     Loss: 10.9262
Epoch: 20/100     Loss: 8.3451
Epoch: 30/100     Loss: 6.7141
Epoch: 40/100     Loss: 5.6266
Epoch: 50/100     Loss: 4.8790
Epoch: 60/100     Loss: 4.3457
Epoch: 70/100     Loss: 3.9500
Epoch: 80/100     Loss: 3.6406
Epoch: 90/100     Loss: 3.3900
Epoch: 99/100     Loss: 3.2024
